In [20]:
# loan_irr_debug.py
import pandas as pd
import math
from datetime import datetime
import os, csv

EXCEL_NAME = "Loan IRR Calc Model.xlsx"
OUT_COMPARE = "cashflow_component_comparison_fixed.csv"

# --- financial helpers ---
def pmt(rate, nper, pv):
    if rate == 0:
        return -pv / nper
    return -(rate * pv) / (1 - (1 + rate) ** -nper)

def irr_from_cashflows(cashflows, guess=0.01, tol=1e-10, maxiter=500):
    def npv(r): return sum(cf / ((1 + r) ** i) for i, cf in enumerate(cashflows))
    r = guess
    for _ in range(maxiter):
        f = npv(r)
        df = sum(-i * cf / ((1 + r) ** (i + 1)) for i, cf in enumerate(cashflows))
        if df == 0:
            break
        r_next = r - f / df
        if abs(r_next - r) < tol:
            return r_next
        r = r_next
    return r

def xnpv(rate, cashflows, dates):
    t0 = dates[0]
    return sum(cf / ((1 + rate) ** ((d - t0).days / 365.0)) for cf, d in zip(cashflows, dates))

def xirr(cashflows, dates, guess=0.05, tol=1e-8, maxiter=200):
    r = guess
    for _ in range(maxiter):
        f = xnpv(r, cashflows, dates)
        h = 1e-6
        f1 = xnpv(r + h, cashflows, dates)
        df = (f1 - f) / h
        if df == 0:
            break
        r_next = r - f / df
        if abs(r_next - r) < tol:
            return r_next
        r = r_next
    return r

# --- main ---
def build_schedule_from_excel(path):
    irr_df = pd.read_excel(path, sheet_name="IRR Calculation")
    irr_raw = pd.read_excel(path, sheet_name="IRR Calculation", header=None)
    charged = pd.read_excel(path, sheet_name="Charged Off", header=None)
    prepay = pd.read_excel(path, sheet_name="Prepay", header=None)

    # parse params in B & C (columns 1 and 2 of headerless read)
    params = {}
    for idx,row in irr_raw[[1,2]].dropna(how='all').iterrows():
        params[row[1]] = row[2]

    term = int(params.get('Term', 36))
    coupon = float(params.get('CouponRate', 0.28))
    invested = float(params.get('Invested', 7500))
    recovery_rate = float(params.get('Recovery_Rate', 0.08))
    purchase_premium = float(params.get('Purchase_Premium', 0.0))
    servicing_fee = float(params.get('Servicing_Fee', 0.0))
    earnout_fee = float(params.get('Earnout_Fee', 0.0))
    default_multiplier = float(params.get('Deafult Multiplier = ', 1))
    prepay_multiplier = float(params.get('Prepay Multiplier = ', 1))
    grade = params.get('Grade', 'C4')

    monthly_rate = coupon / 12.0

    # charged-off lookup
    ch_headers = charged.iloc[0].tolist()
    search_label = f"{term}-{grade}"
    target_col = None
    for i, h in enumerate(ch_headers):
        if isinstance(h, str) and h.strip() == search_label:
            target_col = i
            break
    if target_col is None:
        raise ValueError("Cannot find charged off column for: " + search_label)
    default_rates = {int(charged.iloc[i,0]): float(charged.iloc[i,target_col])
                     for i in range(1, charged.shape[0]) if pd.notna(charged.iloc[i,0])}

    # prepay lookup
    prepay_col = None
    for i in range(prepay.shape[1]):
        h = prepay.iloc[1,i]
        if pd.isna(h): continue
        try:
            if (isinstance(h,str) and (h.strip()==f"{term}M" or h.strip()==str(term))) or float(h)==term:
                prepay_col = i; break
        except: pass
    if prepay_col is None:
        for i in range(prepay.shape[1]):
            try:
                if float(prepay.iloc[0,i])==term: prepay_col = i; break
            except: pass
    if prepay_col is None:
        raise ValueError("Cannot find prepay column for term: " + str(term))
    prepay_speeds = {int(prepay.iloc[i,0]): float(prepay.iloc[i, prepay_col])
                     for i in range(2, prepay.shape[0]) if pd.notna(prepay.iloc[i,0])}

    # use model playdates & model Total_CF for comparison
    model_dates = pd.to_datetime(irr_df['Playdate']).tolist()
    model_total_cf = irr_df['Total_CF'].tolist()

    # build schedule with fixes:
    schedule_rows = []
    balance = invested
    remaining_term = term
    # initial outflow - take Excel first Total_CF to be exact
    initial_outflow = float(model_total_cf[0]) if model_total_cf and not pd.isna(model_total_cf[0]) else -(invested * (1 + purchase_premium))
    python_cashflows = [initial_outflow]
    # per-component arrays
    comp_default, comp_prepay, comp_recovery, comp_interest, comp_servicing, comp_earnout, comp_total = ([] for _ in range(7))

    for m in range(1, term + 1):
        prev_balance = balance

        # default first
        default_amount = prev_balance * default_rates.get(m, 0.0) * default_multiplier
        performing_balance = prev_balance - default_amount

        # recompute payment for remaining term on performing balance
        payment = -pmt(monthly_rate, remaining_term, performing_balance) if remaining_term>0 and performing_balance>0 else 0.0

        sched_interest = performing_balance * monthly_rate
        sched_principal = max(0.0, payment - sched_interest)

        # prepay on performing balance
        prepay_amount = performing_balance * prepay_speeds.get(m, 0.0) * prepay_multiplier

        # recovery -- here we use same-month recovery (Excel might lag)
        recovery = default_amount * recovery_rate

        # interest on performing balance
        interest_amount = sched_interest

        # fees on payments only (exclude defaults)
        servicing_cf = (sched_principal + prepay_amount) * servicing_fee
        earnout_cf = (sched_principal + prepay_amount) * earnout_fee

        principal_cf = sched_principal + prepay_amount
        total_cf = principal_cf + interest_amount + recovery - servicing_cf - earnout_cf

        # append schedule row (for debugging)
        row = {
            "Month": m,
            "Balance_Beg": prev_balance,
            "Default": default_amount,
            "Performing_Balance": performing_balance,
            "Payment": payment,
            "Sched_Principal": sched_principal,
            "Sched_Interest": sched_interest,
            "Prepay": prepay_amount,
            "Recovery": recovery,
            "Interest_Amount": interest_amount,
            "Servicing_CF": servicing_cf,
            "Earnout_CF": earnout_cf,
            "Total_CF": total_cf
        }
        schedule_rows.append(row)

        # append to arrays
        comp_default.append(default_amount)
        comp_prepay.append(prepay_amount)
        comp_recovery.append(recovery)
        comp_interest.append(interest_amount)
        comp_servicing.append(servicing_cf)
        comp_earnout.append(earnout_cf)
        comp_total.append(total_cf)
        python_cashflows.append(total_cf)

        # update balance and counters
        balance = performing_balance - sched_principal - prepay_amount
        if balance < 1e-8:
            balance = 0.0
        remaining_term -= 1

    # return everything
    return {
        "schedule_rows": schedule_rows,
        "python_cashflows": python_cashflows,
        "comp_default": comp_default,
        "comp_prepay": comp_prepay,
        "comp_recovery": comp_recovery,
        "comp_interest": comp_interest,
        "comp_servicing": comp_servicing,
        "comp_earnout": comp_earnout,
        "comp_total": comp_total,
        "model_dates": model_dates,
        "model_total_cf": model_total_cf,
        "irr_df": irr_df
    }

def main():
    if not os.path.exists(EXCEL_NAME):
        raise RuntimeError("Put Loan IRR Calc Model.xlsx in current folder and re-run.")

    out = build_schedule_from_excel(EXCEL_NAME)
#     print(out)
    print(pd.DataFrame(out['schedule_rows']))
#     print()

    # build comparison DataFrame aligning months 1..term
    term = len(out["comp_total"])
    model_df = out["irr_df"].iloc[1:1+term].reset_index(drop=True)  # skip initial row
    cmp = pd.DataFrame({
        "Model_Default": model_df["Default"],
        "Python_Default": out["comp_default"],
        "Diff_Default": model_df["Default"] - pd.Series(out["comp_default"]),
        "Model_Prepay": model_df["Prepay"],
        "Python_Prepay": out["comp_prepay"],
        "Diff_Prepay": model_df["Prepay"] - pd.Series(out["comp_prepay"]),
        "Model_Recovery": model_df["Recovery"],
        "Python_Recovery": out["comp_recovery"],
        "Diff_Recovery": model_df["Recovery"] - pd.Series(out["comp_recovery"]),
        "Model_Interest": model_df["Interest_Amount"],
        "Python_Interest": out["comp_interest"],
        "Diff_Interest": model_df["Interest_Amount"] - pd.Series(out["comp_interest"]),
        "Model_Servicing": model_df["Servicing_CF"],
        "Python_Servicing": out["comp_servicing"],
        "Diff_Servicing": model_df["Servicing_CF"] - pd.Series(out["comp_servicing"]),
        "Model_Earnout": model_df["Earnout_CF"],
        "Python_Earnout": out["comp_earnout"],
        "Diff_Earnout": model_df["Earnout_CF"] - pd.Series(out["comp_earnout"]),
        "Model_Total_CF": model_df["Total_CF"],
        "Python_Total_CF": out["comp_total"],
        "Diff_Total_CF": model_df["Total_CF"] - pd.Series(out["comp_total"])
    })

    cmp.to_csv(OUT_COMPARE, index=False)
#     print("Wrote component comparison to:", OUT_COMPARE)
#     print("\nFirst 12 rows comparison:\n")
#     print(cmp.head(12).to_string(float_format="%.6f"))

    # compute IRRs
    monthly_irr_python = irr_from_cashflows(out["python_cashflows"])
    annual_irr_python = (1 + monthly_irr_python) ** 12 - 1

    # also IRR from model Total_CF (periodic) and XIRR (date-weighted)
    model_total_cf = out["model_total_cf"]
    monthly_irr_model = irr_from_cashflows(model_total_cf)
    annual_irr_model = (1 + monthly_irr_model) ** 12 - 1

    # XIRR using model dates
    model_dates = out["model_dates"]
    # convert to python datetimes
    model_dates_py = [pd.to_datetime(d).to_pydatetime() for d in model_dates]
    try:
        annual_xirr = xirr(model_total_cf, model_dates_py)
    except Exception as e:
        annual_xirr = None

    print("\nIRR (cashflows) annualized: {:.6f}%".format(annual_irr_python*100))
#     print("IRR (Excel Total_CF) periodic annualized: {:.6f}%".format(annual_irr_model*100))
#     if annual_xirr is not None:
#         print("XIRR (Excel cashflows/dates) annualized: {:.6f}%".format(annual_xirr*100))
#     else:
#         print("XIRR calculation failed (dates).")

if __name__ == "__main__":
    main()


    Month  Balance_Beg    Default  Performing_Balance     Payment  \
0       1  7500.000000  13.583624         7486.416376  309.668127   
1       2  7287.278323  33.499358         7253.778965  305.554470   
2       3  7046.156200  49.650666         6996.505534  300.360851   
3       4  6782.978134  62.281990         6720.696144  294.287218   
4       5  6503.578474  71.704046         6431.874429  287.521261   
5       6  6213.191620  78.266485         6134.925135  280.235096   
6       7  5916.405462  82.335241         5834.070221  272.583235   
7       8  5617.150087  84.274701         5532.875386  264.701636   
8       9  5318.713732  84.434434         5234.279298  256.707617   
9      10  5023.778585  83.139989         4940.638596  248.700418   
10     11  4734.469887  80.687135         4653.782752  240.762233   
11     12  4452.412912  77.338905         4375.074007  232.959539   
12     13  4178.793446  73.324791         4105.468655  225.344603   
13     14  3914.418469  68.841533 